In [ ]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 47.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=750028d6b5e1845b78c3c305ee47187d8fb2826c100764ecb249a2dcc900b226
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [35]:
import pyspark
#from pyspark.sql.functions import *

#creating the spark session

spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
spark

In [58]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType,TimestampType

schema_ = StructType([ 
                     StructField("X", StringType(),True), \
                     StructField("Y",FloatType(),True), \
                     StructField("RowID",IntegerType(),True), \
                     StructField("CrimeDate",StringType(),True), \
                     StructField("CrimeCode",StringType(),True), \
                     StructField("Location",StringType(),True), \
                     StructField("Description",StringType(),True), \
                     StructField("Inside_Outside", StringType(),True),\
                     StructField("Weapon",StringType(),True), \
                     StructField("Post",StringType(),True), \
                     StructField("District",StringType(),True), \
                     StructField("Neighborhood",StringType(),True), \
                     StructField("Latitude",FloatType(),True), \
                     StructField("Longitude",FloatType(),True), \
                     StructField("GeoLocation",StringType(),True), \
                     StructField("VRIName",StringType(),True), \
                     StructField("Total_Incidents",IntegerType(),True), \
                     StructField("Shape",StringType(),True) \
                     ])

In [46]:
# Defining the path:
spark_file = "/content/Part1_Crime_data.csv"

# Reading the csv with spark object and passing the schema defined above:

spark_df = spark.read.csv(spark_file, header = True, schema = schema_)

#caching the dataframe:
spark_df = spark_df.cache()

# Show the count of the rows
print(" The crimes dataframe has {} records".format(spark_df.count()))

 The crimes dataframe has 513756 records


In [48]:
#printing the schema:

spark_df.printSchema()

root
 |-- X: string (nullable = true)
 |-- Y: float (nullable = true)
 |-- RowID: integer (nullable = true)
 |-- CrimeDate: string (nullable = true)
 |-- CrimeCode: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Inside_Outside: string (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- Post: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Latitude: float (nullable = true)
 |-- Longitude: float (nullable = true)
 |-- GeoLocation: string (nullable = true)
 |-- VRIName: string (nullable = true)
 |-- Total_Incidents: integer (nullable = true)
 |-- Shape: string (nullable = true)



In [50]:
# Display first 5 rows
spark_df.show(5, truncate= False)

+----------------+---------+-----+----------------------+---------+------------------+--------------------+--------------+-------+----+---------+--------------+--------+---------+------------------+-------+---------------+-----+
|X               |Y        |RowID|CrimeDate             |CrimeCode|Location          |Description         |Inside_Outside|Weapon |Post|District |Neighborhood  |Latitude|Longitude|GeoLocation       |VRIName|Total_Incidents|Shape|
+----------------+---------+-----+----------------------+---------+------------------+--------------------+--------------+-------+----+---------+--------------+--------+---------+------------------+-------+---------------+-----+
|1428019.10487147|589532.75|1    |2022/03/05 03:43:00+00|3NF      |1800 FLEET ST     |ROBBERY - STREET    |null          |FIREARM|213 |SOUTHEAST|FELLS POINT   |39.2847 |-76.5913 |(39.2847,-76.5913)|null   |null           |1    |
|1428019.10487147|589532.75|2    |2022/03/05 03:43:00+00|3NF      |1800 FLEET ST    

### ANSWERS FOR THE GIVEN QUESTION:


In [60]:
# 1. What are distinct crime codes?
print("Total Number of distinct crime code are:",spark_df.select('CrimeCode').distinct().count())

spark_df.select('CrimeCode').distinct().sort('CrimeCode').show()

Total Number of distinct crime code are: 85
+---------+
|CrimeCode|
+---------+
|       1A|
|       1K|
|       1O|
|       2A|
|       2B|
|      3AF|
|     3AJF|
|     3AJK|
|     3AJO|
|      3AK|
|      3AO|
|       3B|
|      3BJ|
|       3C|
|      3CF|
|      3CK|
|      3CO|
|       3D|
|       3E|
|      3EF|
+---------+
only showing top 20 rows



In [12]:
# 2. Count the number of crimes by the crime codes and order by the resulting counts in descending order
spark_df.groupBy('CrimeCode').count().orderBy('count', ascending=False).show()

+---------+-----+
|CrimeCode|count|
+---------+-----+
|       4E|91822|
|       6D|68508|
|       5A|43956|
|       7A|40308|
|       6J|27670|
|       6G|26898|
|       6E|24310|
|       6C|23269|
|       4C|22455|
|       5D|14971|
|      3AF|14760|
|       4B|14501|
|       4A|13240|
|       3B|10735|
|       4D| 7222|
|       5B| 6481|
|       9S| 5452|
|       6F| 5058|
|       5C| 4917|
|       6B| 4106|
+---------+-----+
only showing top 20 rows



Ans 2. It can be clearly seens, Crime code 4E has the highest count.

In [57]:
# 3. Which neighborhood had most crimes?
from pyspark.sql.functions import desc
spark_df.groupBy("Neighborhood").count().sort(desc("count")).show()

+--------------------+-----+
|        Neighborhood|count|
+--------------------+-----+
|            DOWNTOWN|17818|
|           FRANKFORD|12062|
|       BELAIR-EDISON|10895|
|            BROOKLYN| 9043|
|              CANTON| 7548|
| SANDTOWN-WINCHESTER| 7430|
|         CHERRY HILL| 7407|
|               UPTON| 6510|
|        MOUNT VERNON| 6160|
|         FELLS POINT| 6015|
|        INNER HARBOR| 5990|
|PATTERSON PARK NE...| 5988|
|WASHINGTON VILLAG...| 5940|
|      HAMILTON HILLS| 5917|
|COLDSTREAM HOMEST...| 5819|
|           MONDAWMIN| 5761|
|      MCELDERRY PARK| 5676|
|CENTRAL PARK HEIGHTS| 5258|
|    CARROLLTON RIDGE| 5034|
|       BROADWAY EAST| 5004|
+--------------------+-----+
only showing top 20 rows



Ans 3. 'Downtown' has the highest crime number that is 17,818

In [26]:
# 4. Which month of the year had most crimes?

from pyspark.sql import functions as F

#Checking the datatype of CrimeDate column again:
#spark_df.select("CrimeDate").dtypes

#Creating another dataframe "Dates" which contains CrimeDate column as timestamp
Dates= spark_df.withColumn("CrimeDate",to_timestamp("CrimeDate", 'yyyy/MM/dd HH:mm:ss+00'))

#Checking the CrimeDate column's datatype:
# Dates.select("CrimeDate").dtypes

#Now selecting only month from the CrimeDate column in "month" column and sorting by number of counts:
Dates.select(F.date_format('CrimeDate','MM').alias('month')).groupby('month').count().sort(desc("count")).show()

+-----+-----+
|month|count|
+-----+-----+
|   08|46322|
|   10|46207|
|   07|45982|
|   05|45035|
|   06|44989|
|   09|44827|
|   01|42225|
|   11|42022|
|   12|41593|
|   04|40012|
|   03|39002|
|   02|35540|
+-----+-----+



Ans 4. 08 being 'August' the month when most of the crimes has happened in the year 2017. 

In [64]:
# 5. What weapons were used? 

# Removing the null values from the column "Weapon":
spark_df = spark_df.na.drop(subset=["Weapon"])

spark_df.select("Weapon").distinct().show()

+-------+
| Weapon|
+-------+
|     NA|
|  HANDS|
|  KNIFE|
|  OTHER|
|   FIRE|
|FIREARM|
+-------+



Ans 5. The different kind of weapons used were NA, Hands, Knife, Other, Fire, Firearms

In [65]:
# 6. Which weapon was used the most?
spark_df.groupBy("Weapon").count().sort(desc("count")).show()

+-------+------+
| Weapon| count|
+-------+------+
|     NA|408269|
|FIREARM| 46217|
|  OTHER| 30293|
|  KNIFE| 19181|
|  HANDS|  7222|
|   FIRE|  2348|
+-------+------+



Ans 6. Weapons with 'NA' category were used the most.

### --------------------------------------------------------------------------------END OF ASSIGNMENT-------------------------------------------------------------------